In [ ]:
!pip install openai

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from openai import OpenAI
from google.colab import userdata
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
client = OpenAI(api_key=userdata.get('chatgpt_api_key'))
sheet_id = userdata.get('q&a_sheet_id')
sh = gc.open_by_key(sheet_id)
worksheet = sh.sheet1
rows = worksheet.get_all_records()

for index, row in enumerate(rows, start=2):
    question = row['Questions']

    if question:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Give only the answer and not a whole sentence."},
                {"role": "user", "content": question}
            ]
        )

        answer = response.choices[0].message.content.strip()

        worksheet.update_cell(index, 2, answer)

        print(f"Question: {question}\nAnswer: {answer}")

print("All questions have been processed.")